In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

pcd_path = "/home/t/Desktop/work/datasets/cwc/geometry/cloud.ply"  # Update this path to your actual PCD file path


#pcd_path = "/home/t/Desktop/work/datasets/output.ply"  # Update this path to your actual PCD file path
pcd = o3d.io.read_point_cloud(pcd_path)

#pcd_path = "/home/t/Desktop/work/datasets/output.ply"  # Update this path to your actual PCD file path
#pcd = o3d.io.read_point_cloud(pcd_path)

#DATANAME = "appartment_cloud.ply"
#pcd = o3d.io.read_point_cloud(DATANAME)


: 

In [ ]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

nn = 50
std_multiplier = 0.3

filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)

outliers = pcd.select_by_index(filtered_pcd[1], invert=True)
outliers.paint_uniform_color([1, 0, 0])
filtered_pcd = filtered_pcd[0]
print(pcd)

#o3d.visualization.draw_geometries([filtered_pcd, outliers])

: 

In [ ]:
voxel_size= 0.05

pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)
#o3d.visualization.draw_geometries([pcd_downsampled])

: 

In [ ]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
#o3d.visualization.draw_geometries([pcd_downsampled, outliers])

: 

In [ ]:
front = [ -0.013937830691568774, 0.092337126925194149, -0.99563024856962534 ]
lookat = [ 0.1155990791357997, -0.28634510183152262, -0.010700327508947227 ]
up = [ -0.045705399616724998, -0.99474502847220936, -0.09161519947966619 ]
zoom = 0.69999999999999996
pcd = pcd_downsampled
#o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

: 

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.05 ,ransac_n=3, num_iterations=10000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], zoom=zoom, front=front, lookat=lookat, up=up)

: 

In [ ]:
max_plane_idx = 12
pt_to_plane_dist = 0.04

segment_models = {}
segments = {}
rest = pcd

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=20000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)

o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], zoom=zoom, front=front, lookat=lookat, up=up)

: 

In [ ]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()

colors= plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], zoom=zoom, front=front, lookat=lookat, up=up)


: 